In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import inspect_ai
from tqdm.notebook import tqdm
from huggingface_hub import login
import torch as t
import pynvml
import gc
import pandas as pd
from pprint import pprint
import os
from dotenv import load_dotenv

device = 'cuda' if t.cuda.is_available() else 'cpu'

In [2]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Eval Model

In [3]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
import re

In [4]:
# repeat each problem n times
NUM_EPOCHS = 1

# timeout for verification/scoring
VERIFY_TIMEOUT = 30

# instruction prepended to code problem
INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [5]:
@scorer(metrics=[accuracy(), stderr()])
def verify() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        # extract answer from completion and format for verification
        answer = find_code(state.output.completion)

        code = [
            state.metadata["prompt"],
            answer,
            "\n",
            state.metadata["test"],
            "\n",
            "".join(["check(", state.metadata["entry_point"], ")"]),
        ]

        # verify (returns error status if incorrect)
        try:
            result = await sandbox().exec(
                cmd=["python", "-c", "".join(code)],
                timeout=VERIFY_TIMEOUT,
            )
        except TimeoutError:
            result = ExecResult(False, 1, "", "Verification timed out.")

        # return score w/ model answer + scoring details
        return Score(
            value=CORRECT if result.success else INCORRECT,
            answer=answer,
            explanation="".join(
                ["The following verification code was executed:\n\n"]
                + ["```python\n\n"]
                + code
                + ["\n```\n"]
                + [f"\nThe submission was incorrect\n\n{result.stderr}"]
                if not result.success
                else [""]
            ),
        )

    return score


# extract code from completion (removing markdown and/or signature)
def find_code(completion: str) -> str:
    pattern_1 = re.compile(r"```python\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if matches:
        extracted_answer = matches[0]
        extracted_answer = extract_function_body(extracted_answer)
    else:
        extracted_answer = completion
    # print(f'extracted answer: \n{str(extracted_answer)}')
    return str(extracted_answer)


def extract_function_body(code: str) -> str:
    return code[code.find(":\n    ") + 2 :]

In [6]:
lang = 'python'

def humaneval_record_to_sample(record):
    model_input = INSTRUCTION + LANG_PREFIX[lang] + '\n' + record['prompt'] 
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata={
            "prompt": record["prompt"],
            "test": record["test"],
            "entry_point": record["entry_point"],
        },
    )

humaneval_dataset = hf_dataset(
    path = 'openai_humaneval',
    # path = 'THUDM/humaneval-x',
    # name = lang,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

In [11]:
@task
def humaneval():
    return Task(
        dataset = humaneval_dataset,
        solver = generate(),
        scorer = verify(),
        sandbox = 'local',
    )

In [12]:
# model = get_model(
#         'hf/meta-llama/Llama-3.1-8B-Instruct', 
#         device = 'auto',
#         torch_dtype=t.bfloat16,
# )
inspect_ai.eval(humaneval(), model = 'openai/gpt-4o-mini', epochs = NUM_EPOCHS)

Output()

┌────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────┐
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/_eval/task/run.py:799 in task_run_sample │
│                                                                                                                │
│ /tmp/ipykernel_12732/640181400.py:18 in score                                                                  │
│                                                                                                                │
│   15 │   │                                                                                                     │
│   16 │   │   # verify (returns error status if incorrect)                                                      │
│   17 │   │   try:                                                                                              │
│ > 18 │   │   │   result = await sandbox().exec(                                                                │
│   19 │   │   │   │   cmd=["python", "-c", "".join(code)],                                                      │
│   20 │   │   │   │   timeout=VERIFY_TIMEOUT,                                                                   │
│   21 │   │   │   )                                                                                             │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/util/_sandbox/events.py:43 in exec       │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/util/_sandbox/local.py:70 in exec        │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/util/_subprocess.py:199 in subprocess    │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/util/_subprocess.py:191 in               │
│ run_command_timeout                                                                                            │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/util/_subprocess.py:117 in run_command   │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/anyio/_core/_subprocesses.py:190 in open_process    │
│                                                                                                                │
│   187 │   if umask >= 0:                                                                                       │
│   188 │   │   kwargs["umask"] = umask                                                                          │
│   189 │                                                                                                        │
│ > 190 │   return await get_async_backend().open_process(                                                       │
│   191 │   │   command,                                                                                         │
│   192 │   │   stdin=stdin,                                                                                     │
│   193 │   │   stdout=stdout,                                                                                   │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:2

┌────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────┐
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/_eval/task/run.py:370 in task_run        │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/inspect_ai/_util/_async.py:57 in tg_collect         │
│                                                                                                                │
│ /root/miniconda3/envs/srf-env/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:772 in __aexit__        │
│                                                                                                                │
│    769 │   │   │   │   │   # added to self._exceptions so it's ok to break exception                           │
│    770 │   │   │   │   │   # chaining and avoid adding a "During handling of above..."                         │
│    771 │   │   │   │   │   # for each nesting level.                                                           │
│ >  772 │   │   │   │   │   raise BaseExceptionGroup(                                                           │
│    773 │   │   │   │   │   │   "unhandled errors in a TaskGroup", self._exceptions                             │
│    774 │   │   │   │   │   ) from None                                                                         │
│    775 │   │   │   │   elif exc_val:                                                                           │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘
BaseExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)

KeyboardInterrupt: 

### GPU Deets

In [8]:
# Imports
free_memory, total_memory = t.cuda.mem_get_info()

# Convert bytes to GB
free_memory_gb = free_memory / (1024 * 1024 * 1024)
total_memory_gb = total_memory / (1024 * 1024 * 1024)
mem_used = t.cuda.device_memory_used() / (1024 ** 3)

print(f"Free GPU Memory: {free_memory_gb:.2f} GB")
print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
print(f'Memory Used: {mem_used:.2f} GB')

Free GPU Memory: 47.17 GB
Total GPU Memory: 47.43 GB
Memory Used: 0.82 GB


In [18]:
t.cuda.get_device_properties()

_CudaDeviceProperties(name='NVIDIA RTX A6000', major=8, minor=6, total_memory=48566MB, multi_processor_count=84, uuid=63cca492-796d-c7cf-69cf-c5dd0a3d9c8d, L2_cache_size=6MB)

In [13]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")

15316.50830078125 MB allocated
15318.0 MB reserved


In [13]:
# del model, tokenizer
t.cuda.empty_cache()
gc.collect()
# clean()

2215